# Using Cohere Connectors For RAG

This notebook covers how to get started with [Cohere connectors](https://docs.cohere.com/docs/connectors)

## Installation

In [ ]:
# install package
!pip install -U langchain-cohere langchain-core

## Environment Setup

Make sure to set the following environment variables:

- TODO: fill out relevant environment variables or secrets

## Usage

In [ ]:
from langchain_core.documents import Document
from langchain_core.messages.human import HumanMessage
from langchain_core.prompts.chat import (
    ChatPromptTemplate,
    MessagesPlaceholder,
)
from langchain_core.runnables import RunnablePassthrough
from langchain_cohere.chat_models import ChatCohere

In [ ]:
import os

os.environ["COHERE_API_KEY"] = "API_KEY"
llm = ChatCohere(model_name="command-r")

In [ ]:
def get_connectors(_):
    return ["web-search"]

In [ ]:
def format_input_msgs(input):
    return [
        HumanMessage(
            input["message"],
            additional_kwargs={
                "connectors": input.get("connectors", None),
            },
        )
    ]

prompt = ChatPromptTemplate.from_messages(
    [
        # ("system", "You are a helpful assistant that ..."),
        MessagesPlaceholder("input_msgs"),
    ]
)

In [ ]:
chain = (
    {"message": RunnablePassthrough(), "connectors": get_connectors}
    | RunnablePassthrough()
    | {"input_msgs": format_input_msgs}
    | prompt
    | llm
)

In [ ]:
response = chain.invoke("What is today's movement of the AAPL ticker?")
print(response.content)

## Printing results with citations

In [2]:
"""
HELPER CODE FOR CITATIONS FORMATTING
"""
def insert_citations_in_order(text, citations, documents):
    """
    A helper function to pretty print citations.
    """

    citations_reference = {}
    for index, doc in enumerate(documents):
        citations_reference[index] = doc

    offset = 0
    # Process citations in the order they were provided
    for citation in citations:
        # Adjust start/end with offset
        start, end = citation.start + offset, citation.end + offset
        citation_numbers = []
        for doc_id in citation.document_ids:
            for citation_index, doc in citations_reference.items():
                if doc["id"] == doc_id:
                    citation_numbers.append(citation_index)
        references = "(" + ", ".join("[{}]".format(num) for num in citation_numbers) + ")"
        modification = f'{text[start:end]} {references}'
        # Replace the cited text with its bolded version + placeholder
        text = text[:start] + modification + text[end:]
        # Update the offset for subsequent replacements
        offset += len(modification) - (end - start)

    # Add the citations at the bottom of the text
    text_with_citations = f'{text}'
    citations_reference = ["[{}]: {}".format(x["id"], x["snippet"]) for x in citations_reference.values()]

    return text_with_citations, "\n".join(citations_reference)

In [ ]:
cited_response, citations_reference = insert_citations_in_order(response.content, response.response_metadata["citations"], response.response_metadata["documents"])
print(cited_response)
print("\n")
print("References:")
print(citations_reference)